In [1]:
import numpy as np
from pyfigtree import figtree
from sklearn import preprocessing

In [2]:
def direct_evaluation(S,T,weights,delta):
    checkPot = []
    for target in T:
        # for each source, add to potential at current target
        diff = target-S
        norm = np.linalg.norm(diff, axis=1)
        up = (norm / delta)**2
        potential = np.exp(-up)*weights
        
        cum_potential = np.sum(potential)
        checkPot.append(cum_potential)
    return np.array(checkPot)

In [3]:
delta = 1


#Direct Evaluation Test with sphere data
X = np.loadtxt('data/sphere_x.txt')
Y = np.loadtxt('data/sphere_y.txt')
Z = np.loadtxt('data/sphere_z.txt')

X0 = np.dstack((X,Y,Z))[0]

X = np.loadtxt('data/alpha0_x.txt')
Y = np.loadtxt('data/alpha0_y.txt')
Z = np.loadtxt('data/alpha0_z.txt')

alpha0 = np.dstack((X,Y,Z))[0]
weights = alpha0

In [4]:
#Calculating direct Evaluation
cp0_x = direct_evaluation(X0, X0, alpha0[:,0], delta)
cp0_y = direct_evaluation(X0, X0, alpha0[:,1], delta)
cp0_z = direct_evaluation(X0, X0, alpha0[:,2], delta)
cp0 = np.dstack([cp0_x,cp0_y, cp0_z])[0]

In [5]:
cp_x = figtree(X0,X0,alpha0[:,0], delta,epsilon=1e-16, eval="auto")
cp_y = figtree(X0,X0,alpha0[:,1], delta,epsilon=1e-16, eval="auto")
cp_z = figtree(X0,X0,alpha0[:,2], delta,epsilon=1e-16, eval="auto")
cp = np.dstack([cp_x,cp_y, cp_z])[0]

# Form Evolution Equations
## Direct Functions

In [40]:
def direct_position(X,alpha,delta):
    X_x = direct_evaluation(X,X,alpha[:,0], delta)
    X_y = direct_evaluation(X,X,alpha[:,1], delta)
    X_z = direct_evaluation(X,X,alpha[:,2], delta) 
    return np.dstack([X_x,X_y, X_z])[0]

In [49]:
def direct_momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = direct_evaluation(X,X,first_weights[:,0],delta)
    first_sum_y = direct_evaluation(X,X,first_weights[:,1],delta)
    first_sum_z = direct_evaluation(X,X,first_weights[:,2],delta)
    
    
    second_sum_x = direct_evaluation(X,X,second_weights[:,0],delta)
    second_sum_y = direct_evaluation(X,X,second_weights[:,1],delta)
    second_sum_z = direct_evaluation(X,X,second_weights[:,2],delta)
    
    
    third_sum_x = direct_evaluation(X,X,third_weights[:,0],delta)
    third_sum_y = direct_evaluation(X,X,third_weights[:,1],delta)
    third_sum_z = direct_evaluation(X,X,third_weights[:,2],delta)
    
    fourth_sum = direct_evaluation(X,X,ax,delta)
    fifth_sum = direct_evaluation(X,X,ay,delta)
    sixth_sum = direct_evaluation(X,X,az,delta)
    
    second_term = fourth_sum + fifth_sum + sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
    
    result_x = ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) 
    result_y = ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) 
    result_z = ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z)
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

## Figtree Functions

In [33]:
def position(X,alpha,delata):
    X_x = figtree(X,X,alpha[:,0], delta,epsilon=1e-16, eval="auto")
    X_y = figtree(X,X,alpha[:,1], delta,epsilon=1e-16, eval="auto")
    X_z = figtree(X,X,alpha[:,2], delta,epsilon=1e-16, eval="auto") 
    return np.dstack([X_x,X_y, X_z])[0]

In [34]:
def momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = figtree(X,X,first_weights[:,0],delta, epsilon=1e-16, eval="auto")
    first_sum_y = figtree(X,X,first_weights[:,1],delta, epsilon=1e-16, eval="auto")
    first_sum_z = figtree(X,X,first_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    second_sum_x = figtree(X,X,second_weights[:,0],delta, epsilon=1e-16, eval="auto")
    second_sum_y = figtree(X,X,second_weights[:,1],delta, epsilon=1e-16, eval="auto")
    second_sum_z = figtree(X,X,second_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    third_sum_x = figtree(X,X,third_weights[:,0],delta, epsilon=1e-16, eval="auto")
    third_sum_y = figtree(X,X,third_weights[:,1],delta, epsilon=1e-16, eval="auto")
    third_sum_z = figtree(X,X,third_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    fourth_sum = figtree(X,X,ax,delta, epsilon=1e-16, eval="auto")
    fifth_sum = figtree(X,X,ay,delta, epsilon=1e-16, eval="auto")
    sixth_sum = figtree(X,X,az,delta, epsilon=1e-16, eval="auto")
    
    second_term = fourth_sum + fifth_sum + sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
    
    result_x = ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) 
    result_y = ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) 
    result_z = ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z)
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

## Testing

In [58]:
delta = 1

In [59]:
%time postition1 = direct_position(X0,alpha0,delta)

CPU times: user 4.36 s, sys: 0 ns, total: 4.36 s
Wall time: 4.36 s


In [60]:
%time postition2 = position(X0,alpha0,delta)

CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 192 ms


In [61]:
%time momentum1 = direct_momentum(X0,alpha0,delta)

CPU times: user 17.5 s, sys: 12 ms, total: 17.5 s
Wall time: 17.4 s


In [62]:
%time momentum2 = momentum(X0,alpha0,delta)

CPU times: user 764 ms, sys: 0 ns, total: 764 ms
Wall time: 764 ms


In [64]:
error_pos = np.sum(np.abs(postition1-postition2))
error_pos

4.2453886568789216e-13

In [65]:
error_mom = np.sum(np.abs(momentum1-momentum2))
error_mom

5.0976273549552253e-11